In [ ]:
from google.colab import files
uploaded = files.upload()

Saving u1.base to u1.base
Saving u1.test to u1.test
Saving u.data to u.data


In [ ]:
import pandas as pd
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
df = pd.read_csv("u.data", sep='\t', names=r_cols, encoding='latin-1')

In [ ]:

ratings_dict = {}
ratings = df.groupby('movie_id',as_index=False).mean()
for ind,row in ratings.iterrows():
  ratings_dict[int(row.movie_id)-1]= round(row.rating,2)

In [ ]:
#pivot data to get user_id in rows and movie_id in columns
df = df.pivot(index='user_id',columns='movie_id',values='rating')

In [ ]:
#preprocess data
df = df.fillna(0)
df = df.replace([2, 3, 4, 5], 1)
df.head(10)

movie_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from keras.layers import Input, Dense
from keras.models import Model
import numpy as np

Using TensorFlow backend.


In [ ]:
x_train = np.array(df)

In [ ]:
from keras.layers import LeakyReLU

def autoencode(input_shape):
  input = Input(shape=(input_shape,))
  encoded = Dense(512)(input)
  encoded = LeakyReLU(alpha=0.2)(encoded)
  encoded = Dense(256)(encoded)
  encoded = LeakyReLU(alpha=0.2)(encoded)
  encoded = Dense(16)(encoded)
  encoded = LeakyReLU(alpha=0.2)(encoded)

  decoded = Dense(256)(encoded)
  decoded = LeakyReLU(alpha=0.2)(encoded)
  decoded = Dense(512)(decoded)
  decoded = LeakyReLU(alpha=0.2)(encoded)
  decoded = Dense(input_shape, activation='sigmoid')(decoded)
  autoencoder = Model(input, decoded)
  autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
  return autoencoder

In [ ]:
model = autoencode(x_train.shape[1])
model.fit(x_train, x_train,epochs=10,verbose = True)






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/10





943/943 [==============================] - 1s 2ms/step - loss: 0.4836
Epoch 2/10
943/943 [==============================] - 0s 154us/step - loss: 0.2079
Epoch 3/10
943/943 [==============================] - 0s 155us/step - loss: 0.1780
Epoch 4/10
943/943 [==============================] - 0s 165us/step - loss: 0.1625
Epoch 5/10
943/943 [==============================] - 0s 146us/step - loss: 0.1531
Epoch 6/10
943/943 [==============================] - 0s 145us/step - loss: 0.1474
Epoch 7/10
943/943 [==============================] - 0s 155us/step - loss: 0.1436
Epoch 8/10
943/943 [==============================] - 0s 144us/step - loss: 0.1405
Epoch 9/10
943/943 [==============================] - 0s 166us/step - loss: 0.1378
Epoch 10/10
943/943 [==============================] - 0s 149us/step - loss: 0.1356


In [ ]:
new_recommendations = {}
for ind,user in enumerate(x_train):
  movie_ratings = model.predict(np.array([user]))

  sorted_ratings = list(movie_ratings.argsort()[::-1][0])
  rated_movies = list(np.concatenate(np.argwhere(user==1)))
  new_recommendations[ind+1] = [i for i in sorted_ratings if not(i in rated_movies)][:10]


In [ ]:
output_file = open('pred_autoencoder.txt','w')
for userid in range(1943+1):
  print("User:",userid)
  #recommendations = new_recommendations[ind+1]
  recommendations = list(map(lambda x:[x,ratings_dict[x-1]],new_recommendations[ind+1]))
  #print("item:",recommendations)
  #print({userid:recommendations},file=output_file)


In [ ]:
dfbase = pd.read_csv("u1.base", sep='\t', names=r_cols, encoding='latin-1')
#pivot data to get user_id in rows and movie_id in columns
dfbase = dfbase.pivot(index='user_id',columns='movie_id',values='rating')
#preprocess data
dfbase = dfbase.fillna(0)
dfbase = dfbase.replace([2, 3, 4, 5], 1)

X_train = np.array(dfbase)

In [ ]:
dftest = pd.read_csv("u1.test", sep='\t', names=r_cols, encoding='latin-1')
#pivot data to get user_id in rows and movie_id in columns
max_user = max(dftest["movie_id"])
dftest = dftest.pivot(index='user_id',columns='movie_id',values='rating')
#preprocess data
dftest = dftest.fillna(0)
dftest = dftest.replace([2, 3, 4, 5], 1)

X_test = np.array(dfbase)

In [ ]:
model_eval = autoencode(X_train.shape[1])
model_eval.fit(X_train, X_train,epochs=10,verbose = True)

Epoch 1/10
943/943 [==============================] - 0s 466us/step - loss: 0.5031
Epoch 2/10
943/943 [==============================] - 0s 168us/step - loss: 0.2010
Epoch 3/10
943/943 [==============================] - 0s 173us/step - loss: 0.1656
Epoch 4/10
943/943 [==============================] - 0s 175us/step - loss: 0.1515
Epoch 5/10
943/943 [==============================] - 0s 170us/step - loss: 0.1439
Epoch 6/10
943/943 [==============================] - 0s 159us/step - loss: 0.1385
Epoch 7/10
943/943 [==============================] - 0s 193us/step - loss: 0.1345
Epoch 8/10
943/943 [==============================] - 0s 168us/step - loss: 0.1316
Epoch 9/10
943/943 [==============================] - 0s 197us/step - loss: 0.1295
Epoch 10/10
943/943 [==============================] - 0s 163us/step - loss: 0.1275


In [ ]:
new_recommendation = {}
for ind,user in enumerate(X_test):
  movie_ratings = model_eval.predict(np.array([user]))
  sorted_ratings = list(movie_ratings.argsort()[::-1][0])
  rated_movies = list(np.concatenate(np.argwhere(user==1)))
  new_recommendation[ind+1] = [i for i in sorted_ratings if not(i in rated_movies)]


In [ ]:
new_recommendation_eval = {}
for ind,user in enumerate(X_test):
  movie_ratings_eval = np.array([X_test[ind]])
  sorted_ratings_eval = list(movie_ratings_eval.argsort()[::-1][0])
  rated_movies_eval = list(np.concatenate(np.argwhere(user==1)))
  new_recommendation_eval[ind+1] = [i for i in sorted_ratings_eval if not(i in rated_movies_eval)]

In [ ]:
def precision_recall(recommendations,recommendations_test):
  precision,recall = {},{}
  for ind in recommendations:
    a = set(recommendations[ind][:10])
    b = set(recommendations_test[ind][:10])
    c = a.intersection(b)
    precision[ind] = len(c)/len(a)
    recall[ind] = len(c)/(1682-1591)
  return precision,recall      

In [ ]:
precision,recall = precision_recall(new_recommendation,new_recommendation_eval)
print("Average Precision:",sum(precision.values())/len(precision.values()))
print("Average Recall:",sum(recall.values())/len(recall.values()))

Average Precision: 0.004135737009544011
Average Recall: 0.0004544765944553857
